This notebook performs a comprehensive becnhmark suit for the inference procedure for the double pendulum system using the RxInfer framework.

_Author: Dmitry Bagaev_

In [1]:
using DrWatson

In [2]:
@quickactivate "RxInferThesisExperiments"

In [3]:
using RxInferThesisExperiments, ForneyLab, StaticArrays, Plots, PGFPlotsX, LaTeXStrings
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, Dates, DataFrames

In [4]:
const bfolder = datadir("lds", "forneylab")

"/Users/bvdmitri/.julia/dev/thesis/data/lds/forneylab"

In [5]:
# Pregenerate paths for benchmark data
mkpath(bfolder);

In [24]:
# Include the model specification
include(srcdir("models", "forneylab", "rotating.jl"));

In [25]:
function run_benchmark(params)
    @unpack T, d, seed = params
    
    environment = RotatingTracking(d; rng = StableRNG(seed))
    
    states, observations = rand(StableRNG(seed), environment, T);
    model    = rotating(T, seed, environment) 
    result   = run_inference(model, observations)
    e_states = extract_posteriors(T, result)
    amse     = compute_amse(states, e_states)
    
    benchmark_modelcreation = @benchmark rotating($T, $seed, $environment; force = true) 
    
    benchmark_inference = @benchmark run_inference($model, $observations)
    
    emse = compute_emse(seed) do _seed
        local environment = RotatingTracking(d; rng = StableRNG(_seed))
        local states, observations = rand(StableRNG(_seed), environment, T);
        local model    = rotating(T, _seed, environment) 
        local result   = run_inference(model, observations)
        local e_states = extract_posteriors(T, result)
        return compute_amse(states, e_states)
    end
    
    output = @strdict T d seed states e_states observations amse emse benchmark_modelcreation benchmark_inference
    
    return output
end

run_benchmark (generic function with 1 method)

In [26]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params = dict_list(Dict(
    "T"    => [ 10, 20, 30 ],
    "d"    => [ 2, 3, 4 ],
    "seed" => [ 42 ]
));

In [27]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks = map(benchmark_params) do params
    result, _ = produce_or_load(run_benchmark, bfolder, params; tag = false, force = true)
    return result
end;

[ Info: Producing file /Users/bvdmitri/.julia/dev/thesis/data/lds/forneylab/T=10_d=2_seed=42.jld2 now...
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/lds/forneylab/T=10_d=2_seed=42.jld2 saved.
[ Info: Producing file /Users/bvdmitri/.julia/dev/thesis/data/lds/forneylab/T=20_d=2_seed=42.jld2 now...
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/lds/forneylab/T=20_d=2_seed=42.jld2 saved.
[ Info: Producing file /Users/bvdmitri/.julia/dev/thesis/data/lds/forneylab/T=30_d=2_seed=42.jld2 now...
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/lds/forneylab/T=30_d=2_seed=42.jld2 saved.
[ Info: Producing file /Users/bvdmitri/.julia/dev/thesis/data/lds/forneylab/T=10_d=3_seed=42.jld2 now...


LoadError: InterruptException:

In [20]:
sort(prepare_benchmarks_table(bfolder), [ :T ])

[ Info: Scanning folder /Users/bvdmitri/.julia/dev/thesis/data/lds/forneylab for result files.
[ Info: Added 9 entries.


Row,T,seed,amse,emse,d,inference,creation
,Int64?,Int64?,Float64?,Float64?,Int64?,Tuple…?,Tuple…?
1,10,42,2.86933,3.34099,2,"(246658.0, 3.54007e5, 0.0)","(5.48485e9, 5.48485e9, 2.40328e7)"
2,10,42,6.20449,9.30193,3,"(231670.0, 3.04113e5, 0.0)","(4.95792e9, 4.9685e9, 2.92848e7)"
3,10,42,8.26804,27.9913,4,"(256789.0, 3.29563e5, 0.0)","(4.96892e9, 5.00742e9, 3.03118e7)"
4,20,42,2.39579,4.41848,2,"(459081.0, 5.89441e5, 0.0)","(9.54419e9, 9.54419e9, 7.0594e7)"
5,20,42,5.79453,16.7357,3,"(468828.0, 6.00947e5, 0.0)","(9.58611e9, 9.58611e9, 6.61324e7)"
6,20,42,9.47486,45.0845,4,"(517805.0, 6.57436e5, 0.0)","(9.63786e9, 9.63786e9, 7.94159e7)"
7,30,42,2.53305,6.00106,2,"(720007.0, 9.0164e5, 0.0)","(1.43328e10, 1.43328e10, 1.02455e8)"
8,30,42,5.46543,21.7897,3,"(704461.0, 9.10729e5, 0.0)","(1.44209e10, 1.44209e10, 1.12891e8)"
9,30,42,9.16529,48.6285,4,"(784108.0, 9.99046e5, 0.0)","(1.44646e10, 1.44646e10, 1.05846e8)"


# Versions

In [11]:
versioninfo()

Julia Version 1.9.0
Commit 8e630552924 (2023-05-07 11:25 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin22.4.0)
  CPU: 12 × Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, skylake)
  Threads: 2 on 12 virtual cores


In [12]:
] status

Project RxInferThesisExperiments v1.0.0
Status `~/.julia/dev/thesis/Project.toml`
  [b5ca4192] AdvancedVI v0.2.3
  [6e4b80f9] BenchmarkTools v1.3.2
  [76274a88] Bijectors v0.12.4
  [35d6a980] ColorSchemes v3.21.0
  [a93c6f00] DataFrames v1.5.0
  [31c24e10] Distributions v0.25.95
  [634d3b9d] DrWatson v2.12.5
  [442a2c76] FastGaussQuadrature v0.5.1
  [9fc3f58a] ForneyLab v0.12.0
  [f6369f11] ForwardDiff v0.10.35
  [14197337] GenericLinearAlgebra v0.3.11
  [19dc6840] HCubature v1.5.1
  [7073ff75] IJulia v1.24.0
  [b964fa9f] LaTeXStrings v1.3.0
  [bdcacae8] LoopVectorization v0.12.159
  [3bd65402] Optimisers v0.2.18
  [8314cec4] PGFPlotsX v1.6.0
  [e4faabce] PProf v2.2.2
  [91a5bcdd] Plots v1.38.15
  [92933f4c] ProgressMeter v1.7.2
  [37e2e3b7] ReverseDiff v1.14.6
  [86711068] RxInfer v2.11.0
  [860ef19b] StableRNGs v1.0.0
  [aedffcd0] Static v0.8.7
  [90137ffa] StaticArrays v1.5.25
⌅ [2913bbd2] StatsBase v0.33.21
  [4c63d2b9] StatsFuns v1.3.0
  [fce5fe82] Turing v0.25.1
  [37e2e46d] Line